In [1]:
import os

In [2]:
os.environ["NEEDLE_API_KEY"] = ""

In [3]:
os.environ["OPENAI_API_KEY"] = ""

# Document Store

In [35]:
from needle_haystack import NeedleDocumentStore, NeedleEmbeddingRetriever
document_store = NeedleDocumentStore(collection_id='clt_01J5QT9EQ818A1E8GABBPDY0AR') # Berlin City Assistant
document_store.count_documents()

1791

### Test DocumentStore Serialization

In [6]:
ds_serialized = document_store.to_dict()
document_store_deserialized = NeedleDocumentStore.from_dict(ds_serialized)
document_store_deserialized.count_documents()

1791

# NeedleEmbeddingRetriever

In [30]:
prompt = "I just moved in my new apartment. What do i need to do now?"

In [31]:
retriever = NeedleEmbeddingRetriever(document_store=document_store)
result = retriever.run(text=prompt)
print(result)

{'documents': [Document(id=b75fbb45606c3b9308d846f44818e4945eb487ec630c1a08437db26c63fc5ac0, content: 'How to move into an apartment in Germany \- All About Berlin

[All About Berlin](https://allaboutber...', meta: {'file_id': 'fle_01J5R2VS009EGZ9W7EGD2AQTKG'}), Document(id=e03297a3a40e066d4e5e31e99b9c6c0b748f617786d10d6daa89e72e3c37e0af, content: 'Use them to apply for the apartment during the visit. Keep the originals.

**Combine your documents*...', meta: {'file_id': 'fle_01J5R2VRZXGN2FBR5WN5TVV26V'}), Document(id=c993e3eac9cea6d2c9aa0db2239a56da2ab80ac73ecd2e73c0dddc1b4e024e06, content: 'On this page
------------

1. [Before the move](#before-the-move)
	1. [Visit the apartment](#visit-t...', meta: {'file_id': 'fle_01J5R2VS009EGZ9W7EGD2AQTKG'}), Document(id=c1b116150209c970b47e1402085e66f4ef5e18a8f49bd8b32c7adaba65497b95, content: 'How to move out of an apartment in Germany \- All About Berlin

[All About Berlin](https://allaboutb...', meta: {'file_id': 'fle_01J5R2VS019N0KEDQ7CVX5GE

### Test NeedleEmbeddingRetriever Serialization

In [32]:
r_serialized = retriever.to_dict()
r_serialized
retriever_deserialized = NeedleEmbeddingRetriever.from_dict(r_serialized)
result = retriever_deserialized.run(text=prompt)
print(result)

{'documents': [Document(id=b75fbb45606c3b9308d846f44818e4945eb487ec630c1a08437db26c63fc5ac0, content: 'How to move into an apartment in Germany \- All About Berlin

[All About Berlin](https://allaboutber...', meta: {'file_id': 'fle_01J5R2VS009EGZ9W7EGD2AQTKG'}), Document(id=e03297a3a40e066d4e5e31e99b9c6c0b748f617786d10d6daa89e72e3c37e0af, content: 'Use them to apply for the apartment during the visit. Keep the originals.

**Combine your documents*...', meta: {'file_id': 'fle_01J5R2VRZXGN2FBR5WN5TVV26V'}), Document(id=c993e3eac9cea6d2c9aa0db2239a56da2ab80ac73ecd2e73c0dddc1b4e024e06, content: 'On this page
------------

1. [Before the move](#before-the-move)
	1. [Visit the apartment](#visit-t...', meta: {'file_id': 'fle_01J5R2VS009EGZ9W7EGD2AQTKG'}), Document(id=c1b116150209c970b47e1402085e66f4ef5e18a8f49bd8b32c7adaba65497b95, content: 'How to move out of an apartment in Germany \- All About Berlin

[All About Berlin](https://allaboutb...', meta: {'file_id': 'fle_01J5R2VS019N0KEDQ7CVX5GE

{'documents': [Document(id=5eed47be802ba7b4204abc8a534a0d2a14face8a225f53ee6c71287f0fa5b889, content: '* [Business \& Sales](https://corporate.dw.com/de/business-sales/s-3086 "Externer Link — Business & ...', meta: {'file_id': 'fle_01J5ZPM5VGJ3RSXM4KT8KS4DXR'}), Document(id=d479659c22a6aeb0e8c386112fff5889e9f2a272c1d80dbbda3e9e7d3038f3c5, content: 'Pakistans neuer Nationalheld: Speerwurf\-Olympiasieger Nadeem – DW – 09\.08\.2024

You need to enabl...', meta: {'file_id': 'fle_01J5ZPM5VGJ3RSXM4KT8KS4DXR'})]}


In [33]:
from haystack import Pipeline
from haystack.utils import Secret
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders import PromptBuilder  # Correct import for PromptBuilder

# Set up a prompt template for the PromptBuilder
prompt_template = """
Given the following retrieved documents, generate a concise and informative answer to the query:

Query: {{query}}
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Answer:
"""

# Initialize the PromptBuilder
prompt_builder = PromptBuilder(template=prompt_template)

# Initialize the OpenAIGenerator with the API key
llm = OpenAIGenerator()

# Build the pipeline
basic_rag_pipeline = Pipeline()
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", llm)

# Connect the components
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - retriever: NeedleEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [34]:
# Run the full pipeline
result = basic_rag_pipeline.run({
    "retriever": {"text": prompt},
    "prompt_builder": {"query": prompt}
})

# Print the final result generated by the LLM
print("Final generated result:")

print(result['llm']['replies'][0])

Final generated result:
After moving into your new apartment, you should first visit the apartment, sign the lease, get the keys, pay the deposit, and set up an internet contract. After moving in, remember to put your name on the mailbox, register your address, get an electricity contract, buy furniture and appliances, install light fixtures, change your address, get liability insurance, defend your rights, and lower your rent. Additionally, make sure to stop junk mail, deduct your moving costs, and consider getting a PDF merging tool to organize your documents for the landlord.
